# Project 2

It is October 2018. The squirrels in Central Park are running into a problem and we need your help.

For this project you must go through most steps in the checklist. You must write responses for all items however sometimes the item will simply be "does not apply". Some of the parts are a bit more nebulous and you simply show that you have done things in general (and the order doesn't really matter). Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Do not do the final part (launching the product) and your presentation will be done as information written in this document in a dedicated section, no slides or anything like that. It should however include the best summary plots/graphics/data points.

You are intentionally given very little information thus far. You must communicate with your client (me) for additional information as necessary. But also make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself - you should continuously ask questions for getting help).

You must submit all data files and a pickled preprocessor and final model along with this notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Frame the Problem and Look at the Big Picture
========================

1. **Define the objective in business terms.**
   - The objective is to develop a predictive model to identify potentially sick squirrels in Central Park from reported sightings, enabling the client to respond quickly to disease risks. The model should help prevent squirrel disease spread, preserving this critical ecological component by prioritizing intervention for identified sick individuals.

2. **How will your solution be used?**
   - The model will provide online predictions, flagging sightings that likely indicate sickness based on specific patterns. These flagged sightings will trigger response actions, helping the client or park rangers identify which squirrels need attention, ideally catching every sick squirrel to mitigate any spread.

3. **What are the current solutions/workarounds (if any)?**
   - Currently, sightings are reported by people, but there's no predictive mechanism to evaluate the health status of observed squirrels. Without this model, decisions might rely on manual assessments or sporadic checks, which may not consistently or accurately detect health risks.

4. **How should you frame this problem (supervised/unsupervised, online/offline, …)?**
   - This is a **supervised, binary classification** problem. We aim to classify sightings as "sick" or "not sick," based on labeled data or inferred from characteristics. Since the client needs ongoing predictions, an **online** learning model would be appropriate if data is continuously fed.

5. **How should performance be measured? Is the performance measure aligned with the business objective?**
   - **Recall** (sensitivity) should be prioritized, as the client aims to catch every sick squirrel, even at the cost of some false positives. High recall aligns with the business goal of minimizing undetected cases, thereby controlling disease spread.

6. **What would be the minimum performance needed to reach the business objective?**
   - A minimum recall threshold of perhaps **90-95%** could be necessary, given the critical nature of detecting all potential sickness cases. The model may tolerate some level of false positives, but it should rarely miss any cases of sick squirrels.

7. **What are comparable problems? Can you reuse experience or tools?**
   - This problem is similar to **medical screening** or **wildlife monitoring** models, where high recall is critical to prevent issues like disease outbreaks. Techniques used in health diagnostics or ecological monitoring, like anomaly detection, high-recall classifiers, and time-series analysis, might be applicable.

8. **Is human expertise available?**
   - Not as of currently.

9. **How would you solve the problem manually?**
   - I would focus on specific symptoms and actions from the squirrels. I would also focus on working with weather data as weather could play a part in squirrel sickness.

10. **List the assumptions you (or others) have made so far. Verify assumptions if possible.**
    - **Assumptions**:
      - The sightings data includes identifiable indicators of sickness.


Get the Data
=======================================================================

1. **List the data you need and how much you need.** 
    - We need data to predict either sick, or not sick. To do that we need New York Central Park squirrel census from 2018. We would also need label data to know whether the squirrels are sick or not. We would also need weather data to identify any correlation. 
2. **Find and document where you can get that data:** 
    - From the client the diseased squirrels dataset. 
    - `https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw/about_data`
3. **Get access authorizations:** 
    - Done
4. **Create a workspace (with enough storage space):**
    - This notebook.
5. **Get the data:** 
    - Got the data
6. **Convert the data to a format you can easily manipulate (without changing the data itself):** 
    - Done, it's a CSV.
7. **Ensure sensitive information is deleted or protected (e.g. anonymized):** 
    - Done
8. **Check the size and type of data (time series, geographical, …):** 
    - <mark>TODO</mark>
9. **Sample a test set, put it aside, and never look at it (no data snooping!):** 
    - <mark>TODO</mark> 

In [11]:
def merge_data():
    # read data
    df1 = pd.read_csv('diseased_squirrels.csv', header=None, names=['Unique Squirrel ID'])
    df2 = pd.read_csv('2018_Central_Park_Squirrel_Census_-_Squirrel_Data_20241104.csv')

    # check column names
    print(df1.columns)
    print(df2.columns)

    # ensure column names match
    if 'Unique Squirrel ID' in df1.columns and 'Unique Squirrel ID' in df2.columns:
        # create a 'Diseased' column in df2
        df2['Diseased'] = df2['Unique Squirrel ID'].isin(df1['Unique Squirrel ID'])
    else:
        raise KeyError("Column 'Unique Squirrel ID' not found in one of the dataframes")
        
    # save data
    df2.to_csv('merged_data.csv', index=False)


In [12]:
merge_data()

Index(['Unique Squirrel ID'], dtype='object')
Index(['X', 'Y', 'Unique Squirrel ID', 'Hectare', 'Shift', 'Date',
       'Hectare Squirrel Number', 'Age', 'Primary Fur Color',
       'Highlight Fur Color', 'Combination of Primary and Highlight Color',
       'Color notes', 'Location', 'Above Ground Sighter Measurement',
       'Specific Location', 'Running', 'Chasing', 'Climbing', 'Eating',
       'Foraging', 'Other Activities', 'Kuks', 'Quaas', 'Moans', 'Tail flags',
       'Tail twitches', 'Approaches', 'Indifferent', 'Runs from',
       'Other Interactions', 'Lat/Long'],
      dtype='object')


In [15]:
squirrel_data = pd.read_csv('merged_data.csv')
squirrel_data.head()

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,...,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long,Diseased
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,...,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086),False
1,-73.968857,40.783783,21B-AM-1019-04,21B,AM,10192018,4,NaN,NaN,NaN,...,False,False,False,False,False,False,False,NaN,POINT (-73.9688574691102 40.7837825208444),False
2,-73.974281,40.775534,11B-PM-1014-08,11B,PM,10142018,8,NaN,Gray,NaN,...,False,False,False,False,False,False,False,NaN,POINT (-73.97428114848522 40.775533619083),False
3,-73.959641,40.790313,32E-PM-1017-14,32E,PM,10172018,14,Adult,Gray,NaN,...,False,False,False,False,False,False,True,NaN,POINT (-73.9596413903948 40.7903128889029),False
4,-73.970268,40.776213,13E-AM-1017-05,13E,AM,10172018,5,Adult,Gray,Cinnamon,...,False,False,False,False,False,False,False,NaN,POINT (-73.9702676472613 40.7762126854894),False


In [16]:
squirrel_data['Diseased'].value_counts()

Diseased
False    2701
True      322
Name: count, dtype: int64

In [17]:
# view date range 
squirrel_data['Date'].min(), squirrel_data['Date'].max()


(np.int64(10062018), np.int64(10202018))

In [19]:
# Creating a DataFrame for the provided data in the desired structure

# Define the column headers
columns = ["Time", "Temperature (°F) Max", "Temperature (°F) Avg", "Temperature (°F) Min",
           "Dew Point (°F) Max", "Dew Point (°F) Avg", "Dew Point (°F) Min",
           "Humidity (%) Max", "Humidity (%) Avg", "Humidity (%) Min",
           "Wind Speed (mph) Max", "Wind Speed (mph) Avg", "Wind Speed (mph) Min",
           "Pressure (in) Max", "Pressure (in) Avg", "Pressure (in) Min",
           "Precipitation (in) Total"]

# Populate data from the provided information
data = [
    [1, 78, 71.5, 65, 66, 61.9, 58, 84, 72.2, 60, 14, 7.3, 3, 30.3, 30.2, 30.2, 0.00],
    [2, 78, 71.3, 66, 68, 64.7, 62, 94, 80.0, 62, 17, 9.2, 0, 30.2, 30.1, 29.9, 0.00],
    [3, 77, 70.5, 67, 63, 59.9, 57, 87, 69.9, 53, 14, 8.3, 3, 30.1, 30.0, 30.0, 0.56],
    [4, 78, 71.7, 64, 70, 63.4, 59, 84, 75.7, 58, 13, 5.6, 0, 30.0, 30.0, 29.9, 0.00],
    [5, 73, 66.5, 61, 66, 54.2, 51, 78, 64.9, 54, 17, 9.4, 0, 30.2, 30.1, 30.0, 0.04],
    [6, 69, 66.1, 64, 63, 58.5, 55, 87, 76.8, 73, 12, 7.8, 3, 30.2, 30.2, 30.1, 0.00],
    [7, 81, 74.7, 70, 69, 66.8, 64, 87, 77.0, 62, 10, 5.6, 0, 30.2, 30.1, 30.1, 0.00],
    [8, 73, 67.6, 65, 68, 63.2, 60, 93, 86.0, 78, 14, 10.2, 5, 30.4, 30.4, 30.3, 0.00],
    [9, 77, 72.4, 67, 68, 66.4, 64, 93, 81.7, 69, 13, 6.0, 0, 30.4, 30.3, 30.2, 0.00],
    [10, 82, 75.1, 71, 70, 68.2, 65, 87, 79.5, 56, 14, 7.9, 3, 30.2, 30.1, 29.9, 0.00],
    [11, 80, 73.9, 72, 73, 69.5, 67, 94, 86.2, 79, 17, 8.9, 0, 29.9, 29.7, 29.5, 0.00],
    [12, 72, 62.5, 54, 68, 49.0, 37, 88, 62.6, 44, 25, 16.5, 3, 29.8, 29.6, 29.4, 1.15],
    [13, 58, 53.3, 49, 43, 39.7, 36, 74, 60.6, 51, 16, 10.9, 6, 30.1, 29.9, 29.8, 0.02],
    [14, 60, 55.8, 52, 42, 38.8, 36, 62, 53.3, 42, 10, 6.8, 0, 30.2, 30.2, 30.1, 0.03],
    [15, 70, 63.2, 55, 63, 56.2, 42, 88, 78.2, 62, 23, 10.4, 0, 30.2, 30.0, 29.8, 0.02],
    [16, 63, 55.0, 49, 54, 35.4, 28, 75, 48.8, 34, 26, 13.9, 7, 30.1, 30.1, 29.9, 0.04],
    [17, 63, 55.8, 50, 40, 35.4, 30, 59, 46.8, 34, 23, 15.8, 7, 30.1, 30.0, 29.8, 0.00],
    [18, 52, 48.5, 44, 31, 25.3, 19, 52, 41.2, 27, 23, 15.2, 6, 30.3, 30.3, 30.1, 0.00],
    [19, 62, 54.3, 47, 43, 32.4, 25, 56, 44.5, 28, 20, 10.8, 7, 30.3, 30.1, 29.9, 0.00],
    [20, 65, 59.1, 55, 51, 45.5, 39, 83, 62.3, 42, 20, 12.1, 6, 29.9, 29.7, 29.6, 0.07],
    [21, 54, 48.4, 43, 45, 32.6, 24, 77, 55.5, 42, 29, 18.3, 8, 30.1, 29.9, 29.6, 0.00],
    [22, 58, 48.7, 43, 36, 30.6, 25, 63, 50.2, 33, 14, 9.9, 6, 30.2, 30.2, 30.1, 0.00],
    [23, 76, 56.0, 47, 43, 39.2, 32, 77, 55.8, 29, 20, 10.5, 3, 30.1, 30.0, 29.9, 0.00],
    [24, 55, 50.8, 47, 37, 34.5, 31, 66, 54.2, 41, 26, 17.9, 12, 30.1, 30.0, 29.9, 0.00],
    [25, 52, 46.9, 43, 34, 30.5, 28, 63, 53.3, 41, 21, 15.2, 10, 30.1, 30.1, 30.0, 0.00],
    [26, 50, 46.1, 42, 37, 31.7, 27, 64, 57.6, 48, 15, 7.5, 0, 30.1, 30.1, 30.0, 0.00],
    [27, 52, 49.7, 45, 50, 46.3, 36, 93, 88.2, 66, 37, 23.0, 8, 30.0, 29.6, 29.5, 0.66],
    [28, 54, 50.1, 47, 46, 42.2, 38, 93, 75.4, 57, 15, 9.6, 5, 29.8, 29.7, 29.6, 0.36],
    [29, 57, 53.3, 49, 48, 42.1, 36, 83, 66.9, 49, 18, 12.6, 6, 29.9, 29.8, 29.6, 0.07],
    [30, 57, 50.8, 45, 34, 30.5, 28, 65, 47.1, 33, 22, 12.9, 0, 30.1, 30.0, 29.9, 0.00], 
    [31, 64, 56.3, 47, 51, 41.3, 28, 72, 57.8, 43, 17, 8.0, 0, 30.1, 30.0, 29.9, 0.00]
]

# Create DataFrame
weather_df = pd.DataFrame(data, columns=columns)

# Saving to a CSV file
file_path = 'October_weather_data.csv'
weather_df.to_csv(file_path, index=False)

file_path


'October_weather_data.csv'